# Covid-19 Daily Tracking


In [14]:
import sys
import os.path
import requests
import json
from datetime import datetime 
import pandas as pd

COUNTRY_LOC = 'malaysia'
API_URL = 'https://api.covid19api.com/total/country/' + COUNTRY_LOC
COVID_LOCAL = '../data/covid_19.json'

start_date = '2021-02-01'
today_date = datetime.today()
url_param = {'from': start_date, 'to': str(today_date)[:10]}

try:
    api_request = requests.get(API_URL, params=url_param, timeout=30)

    if os.path.isfile(COVID_LOCAL):
        print('Exist')
        t = os.path.getmtime(COVID_LOCAL)
        file_date = (str(datetime.fromtimestamp(t)))[:10]

        if file_date < str(today_date)[:10]:
            if api_request.ok:
                print('Outdated File. Updating new data from web.....')
                with open(COVID_LOCAL, "wb") as file:
                    file.write(api_request.content) 
                df = pd.read_json(COVID_LOCAL)
                df.set_index('Date', inplace=True)
            else:
                print('Error: ', api_request.status_code)
                print('Use existing file.....')
                df = pd.read_json(COVID_LOCAL)
                df.set_index('Date', inplace=True)
        else:
            print('Retrieving from current file....')
            df = pd.read_json(COVID_LOCAL)
            df.set_index('Date', inplace=True)
        
    else:
        if api_request.ok:
            print('Pulling data from web to new file....')
            with open(COVID_LOCAL, "wb") as file:
                file.write(api_request.content) 
            df = pd.read_json(COVID_LOCAL)
            df.set_index('Date', inplace=True)
        else:
            print('Error: ', api_request.status_code)
            print('No Data File exist....quiting')
            sys.exit(0)

except requests.exceptions.RequestException as e:
    print('Server Request Failed :', e)
    print('using existing file ....')
    df = pd.read_json(COVID_LOCAL)
    df.set_index('Date', inplace=True)




Exist
Retrieving from current file....


In [ ]:
df

In [ ]:
# remove unwanted Columns
df.drop(columns=['Country', 'CountryCode', 'Province', 'City', 'CityCode', 'Lat', "Lon"], inplace=True)

In [ ]:
# rename columns date to strip of time info
for item in df.index:
    df.rename(index={item: str(item)[:10]}, inplace=True)
df

In [ ]:
# Population by Country data pulled from UN [source: https://population.un.org/wpp/Download/Standard/Population/] edited to conform to country's name
pop_df = pd.read_csv('../data/population.csv')
"""
for index, item in enumerate(pop_df['Pop.(\'000)']):
    # strip empty spaces and convert to integer (in millions)
    pop_df.loc[index, 'Pop.(\'000)'] = int(item.replace(' ', ''))
    pop_df.loc[index, 'Pop.(\'000)'] = pop_df.loc[index, 'Pop.(\'000)'] * 1000 

    # rename Country's name to conform to covid's dataframe
    pop_df.loc[index, 'Country'] = pop_df.loc[index, 'Country'].lower().replace(' ', '-')
"""
pop_df['Country'] = pop_df['Country'].str.replace(' ', '-').str.lower()
pop_df['Pop.(\'000)'] = pop_df['Pop.(\'000)'].str.replace(' ', '').astype(int) \
 * 1000 #pop_df.set_index('Country', inplace=True)
pop_df

## Data Crunching to form new dataset


In [ ]:
daily_cases = []
daily_deaths = []
daily_recovered = []
mortality_rate = []
recovered_rate = []
active_rate = []
cases_per_million_pop = []

def append_rate_data(index):
    """ to append into list after calculation of
            - mortality rate
            - recovered rate
            - active rate
            - cases per 1mil population
    """
    
    # TODO: catch error for division with 0.0 (Not Working)
  
    mortality_rate.append(round((df.loc[df.index[index], 'Deaths'] / df.loc[df.index[index], 'Confirmed']) * 100, 2))
    recovered_rate.append(round((df.loc[df.index[index], 'Recovered'] / df.loc[df.index[index], 'Confirmed']) * 100, 2))
    active_rate.append(round((df.loc[df.index[index], 'Active'] / df.loc[df.index[index], 'Confirmed']) * 100, 2))
    cases_per_million_pop.append(round((df.loc[df.index[index], 'Confirmed'] / pop_df.loc[COUNTRY_LOC, 'Pop.(\'000)']) * 1000000, 0))    

      
for index, item in enumerate(df.index):
    if item == df.index[0]:
        new_case, new_death, new_recovered = df.loc[df.index[0], ['Confirmed', 'Deaths', 'Recovered']] 
        daily_cases.append(new_case)
        daily_deaths.append(new_death)
        daily_recovered.append(new_recovered)

    else:
        new_case, new_death, new_recovered = df.loc[df.index[index], ['Confirmed', 'Deaths', 'Recovered']] \
                                            - df.loc[df.index[index - 1], ['Confirmed', 'Deaths', 'Recovered']]
        daily_cases.append(new_case)
        daily_deaths.append(new_death)
        daily_recovered.append(new_recovered)

    df.loc[item, ['Daily Cases', 'Daily Deaths', 'Daily Recovered']] \
                  = daily_cases[index], daily_deaths[index], daily_recovered[index]

# release unwanted objects from memory
del daily_cases, daily_deaths, daily_recovered


df['Mortality Rates'] = round(df['Deaths'].divide(df ['Confirmed']).fillna(0.0), 2)
df['Recovered Rates'] = round(df['Recovered'].divide(df ['Confirmed']).fillna(0.0), 2)
df['Active Rates'] = round(df['Active'].divide(df ['Confirmed']).fillna(0.0), 2)
df['Cases per 1mil pop'] = round(df['Confirmed'] / pop_df.loc[COUNTRY_LOC, 'Pop.(\'000)'] * 1000000, 2)   

# TODO: add to df column

df


In [13]:
# replace NaN to zero
#df.fillna(0.0, inplace=True)
df.tail(7)

,Confirmed,Deaths,Recovered,Active,Daily Cases,Daily Deaths,Daily Recovered,Mortality Rates,Recovered Rates,Active Rates
Date,,,,,,,,,,
2020-07-29,8956,124,8612,220,13.0,0.0,5.0,0.01,0.96,0.02
2020-07-30,8964,124,8617,223,8.0,0.0,5.0,0.01,0.96,0.02
2020-07-31,8976,125,8644,207,12.0,1.0,27.0,0.01,0.96,0.02
2020-08-01,8985,125,8647,213,9.0,0.0,3.0,0.01,0.96,0.02
2020-08-02,8999,125,8664,210,14.0,0.0,17.0,0.01,0.96,0.02
2020-08-03,9001,125,8668,208,2.0,0.0,4.0,0.01,0.96,0.02
2020-08-04,9002,125,8684,193,1.0,0.0,16.0,0.01,0.96,0.02


In [ ]:
# Max Daily Record
round(df[['Daily Cases', 'Daily Deaths', 'Daily Recovered']].max(), 2)

In [ ]:
round(df[['Daily Cases', 'Daily Deaths', 'Daily Recovered']].mean(), 2)

In [ ]:
last_entry_date = df.index[len(df.index) - 1]
df.loc[last_entry_date]